### Carregamento dos dados

Abaixo carregamos a lista de escolas selecionadas, o shapefile de estados do Brasil e o shapefile de municípios do Brasil

In [ ]:
import numpy as np # álgebra linear
import pandas as pd # processamento de dados
import geopandas as gpd # geoprocessamento

import matplotlib.colors as colors # para definição de cores personalizadas
import matplotlib.pyplot as plt # biblioteca gráfica para visualizarmos os plots

import os

escolas = pd.read_csv('/kaggle/input/censo-2020-escolas/escolas_selecionadas.CSV')
shape_estados = gpd.read_file('/kaggle/input/ibge-municipios-ufs-shape/BR_UF_2020.shp')
shape_municipios = gpd.read_file('/kaggle/input/ibge-municipios-ufs-shape/BR_Municipios_2020.shp')

escolas.head()

In [ ]:
print('Quantidade de escolas em território nacional em 2020:', len(escolas))
print('Quantidade de municípios em 2020:', len(shape_municipios))
print('')

### Densidade de escolas por município

Apenas mostrar a quantidade de escolas por município não seria muito intuitivo, pois algums municípios da região Norte têm uma ampla extensão territorial. Logo,muitas escolas estão nos mesmo municípios.

Assim, fazemos uma pequena adaptação para que seja exibida a densidade de escolas por km². Ou seja, a densidade de escolas naquele município

In [ ]:
# inicializa uma contagem de escolas por município
n_escolas_mun = np.zeros(len(shape_municipios), dtype=int)

# iterativamente, verifica quantas escolas estão registradas naquele município
for i, cod in enumerate(shape_municipios['CD_MUN']):
    n_escolas_mun[i] = (escolas['CO_MUNICIPIO'] == int(cod)).sum()

# definimos "densidade" como a quantidade de escolas por kilômetro quadrado
shape_municipios['n_ies'] = n_escolas_mun
shape_municipios['densidade'] = shape_municipios['n_ies'] / shape_municipios['AREA_KM2']

# usaremos um plot com duas normas, senão, visualizaremos quase todos os municípios com baixa
# densidade em branco, e alguns pontos de destaque
divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.009, vmax=0.08)

# Mostra a densidade
shape_municipios.plot(column='densidade', cmap='Blues', norm=divnorm, edgecolor='#9fe1a1',
                      linewidth=0.1, figsize=(12, 8))

# Desativa eixo e mostra mapa
plt.axis('off')
plt.show()

### Dependência por estado

Na célula abaixo separamos as escolas por estado e também por dependência (Federal, Estadual, Municipal e Privada). Cada escola pertence a uma única dependência.

In [ ]:
# códigos de estado pelo IBGE
codigos_uf = {'Rondônia':11, 'Acre':12, 'Amazonas':13, 'Roraima':14, 'Pará':15, 'Amapá':16, 'Tocantins':17,
 'Maranhão':21, 'Piauí':22, 'Ceará': 23, 'Rio Grande do Norte': 24, 'Paraíba':25, 'Pernambuco':26, 'Alagoas':27, 'Sergipe':28, 'Bahia': 29,
 'Minas Gerais':31, 'Espírito Santo':32, 'Rio de Janeiro':33, 'São Paulo':35,
 'Paraná':41, 'Santa Catarina':42, 'Rio Grande do Sul':43,
 'Mato Grosso do Sul':50, 'Mato Grosso':51, 'Goiás':52, 'Distrito Federal':53}

# Cria dataframe para organizar a quantidade de escolas por estado
por_estado = pd.DataFrame(columns = ['nome', 'federal', 'estadual', 'municipal', 'privada'])
por_estado.index.name = 'codigo_uf'

# tipos de dependência de acordo com os metadados
dependencias = {'federal':1, 'estadual':2, 'municipal':3, 'privada':4}

for nome_uf in codigos_uf:
    # colhe escolas da unidade da federação
    escolas_idx = escolas['CO_UF'] == codigos_uf[nome_uf]
    escolas_uf = escolas[escolas_idx]
    
    # tipo de dependência das escolas
    tp_dependencia = escolas_uf['TP_DEPENDENCIA']
    
    quantidades = []
    for index, dependencia in enumerate(dependencias):
        # retorna código para cada tipo de dependência
        codigo = dependencias[dependencia]

        # coleta a dependência das escolas desta região
        idx_escolas = tp_dependencia == codigo
        escolas_dependencia = escolas_uf[idx_escolas]
        
        qnt = np.unique(escolas_dependencia['CO_ENTIDADE']).size
        quantidades.append(qnt)
    
    # Adiciona nova linha ao nosso dataframe
    row = [nome_uf, quantidades[0], quantidades[1], quantidades[2], quantidades[3]]
    por_estado.loc[codigos_uf[nome_uf]] = row

# criamos uma coluna para somar o total por estado
dependencias = por_estado[['federal', 'estadual', 'municipal', 'privada']]
por_estado['total'] = dependencias.sum(axis=1).astype(int)

# mostra no console
por_estado

### Escolas públicas e privadas por estado

Abaixo criamos uma gráfico que mostra a porcentagem de escolas públicas por estado. Quanto mais escura a cor, maior a porcentagem de escolas públicas na unidade federativa.

In [ ]:
# Total de escolas públicas por estado
tot_publico = np.array(por_estado[['federal', 'estadual', 'municipal']].sum(axis=1))
# total de escolas particulas por estado
tot_privado = np.array(por_estado['privada'])

# colhemos apenas uma casa decimal
shape_estados['perc_publico'] = ((tot_publico / (tot_publico+tot_privado) * 1000)).astype(int) / 10.0

# mostra o mapa em tons de azul
shape_estados.plot(column='perc_publico', cmap='Blues', edgecolor='w', figsize=(12, 8), legend=True,
                   legend_kwds={'label':'Porcentagem de escolas públicas'})

# anota a sigla das regiões no mapa
for idx, row in shape_estados.iterrows():
    plt.annotate(text=row['SIGLA_UF'], xy=row.geometry.representative_point().coords[:][0], fontsize=10,
                 horizontalalignment='center', color='#f8b7cd')

plt.show()

### Distribuição de escolas públicas por dependência

Para se ter uma ideia de como as escolas estão distribuídas pelo país, abaixo executamos o K-Means para colher porções representativas de frequência. Os tamanhos dos círculos são proporcionais à quantidade de escolas na região.

In [ ]:
from sklearn.cluster import KMeans

# Mostra o brasil novamente
ax = shape_estados.plot(color='#d0efff', edgecolor='w', figsize=(12, 8))

# Os tipos de dependência que vamos trabalhar
tipos = ['Federal', 'Estadual', 'Municipal']
cores = ['red', 'green', 'blue']

# Vamos trabalhar com o K-Means para representar essas quantidades que não podem ser
# exibidas integralmente. Aqui estipulamos a quantidade de clusters para cada tipo.
clusters = [35, 55, 95]

# Colhemos as coordenadas x e y dos pontos representativos dos municípios
localizacao = shape_municipios['geometry'].representative_point()
localizacao = np.array([[p.x, p.y] for p in localizacao])

# Criamos um dataframe de mapeamento. Os índices são os códigos dos municípios
mapeamento = pd.DataFrame({'x':localizacao[:,0], 'y':localizacao[:,1]})
mapeamento.index = shape_municipios['CD_MUN'].astype(int)

# Iteramos sobre cada tipo de dependência. Invertemos para que assim as maiores
# bolhas apareçam por baixo das menores
for t in [3, 2, 1]:
    # índices das escolas com essa dependência
    idx_dependencia = escolas['TP_DEPENDENCIA'] == t
    
    # escolas da dependência
    escolas_dependencia = escolas[idx_dependencia]
    
    # A localização dos municípios das escolas
    municipios = escolas_dependencia['CO_MUNICIPIO']
    localizacao = mapeamento.loc[municipios]
    
    # Executamos o K-Means nas posições x e y
    kmeans = KMeans(n_clusters=clusters[t-1], random_state=0).fit(localizacao[['x','y']])
    
    # colhemos os resultados
    closest = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    # quantidade de elementos por cluster
    _, size = np.unique(closest, return_counts=True)
    
    # mostra no mapa as densidades de escolas por tipo. Observe que o tamanho
    # dos círculos não segue uma distribuição linear.
    ax.scatter(centroids[:,0], centroids[:,1], s=(size**0.8)*2, c=cores[t-1],
               alpha=0.6, edgecolor='w')

plt.axis('off')
plt.show()